In [1]:
import pandas as pd 
import numpy as np


In [2]:
file = r'C:\Users\tejag\OneDrive\Desktop\New folder\L27.xlsx'

In [3]:
data = pd.read_excel(file)

In [4]:
data.head()

,km(constant),m/s,N,wt. %,x10-5 mm3/Nm,x10-5 mm3/Nm.1,x10-5 mm3/Nm.2
0,sliding distance,sliding velocity,normal load,filler %,SWR,SWR,SWR
1,5,2,20,4,2.627179,17.275695,6.766977
2,5,4,60,2,3.173835,0.887598,3.631082
3,5,4,60,0,0.109748,0.768239,4.225313
4,5,4,20,0,1.481603,0.65849,12.264384


In [5]:
data.tail()

,km(constant),m/s,N,wt. %,x10-5 mm3/Nm,x10-5 mm3/Nm.1,x10-5 mm3/Nm.2
23,5,4,40,2,4.276608,1.412087,7.100783
24,5,4,60,4,4.484781,6.846589,3.529443
25,5,2,20,0,0.411557,0.823113,6.667215
26,5,6,60,0,0.109748,0.713365,5.020989
27,5,6,40,2,0.847252,1.73485,4.559025


In [6]:
data.isnull().sum()

km(constant)      0
m/s               0
N                 0
wt. %             0
x10-5 mm3/Nm      0
x10-5 mm3/Nm.1    0
x10-5 mm3/Nm.2    0
dtype: int64

In [7]:
# Rename columns
data.rename(columns={
    "km(constant)": "Material_Constant",
    "m/s": "sliding velocity",
    "N": "normal load",
    "wt. %": "filler",
    "x10-5 mm3/Nm": "SWR1",
    "x10-5 mm3/Nm.1": "SWR2",
    "x10-5 mm3/Nm.2": "SWR3"
}, inplace=True)


In [8]:
data.head()

,Material_Constant,sliding velocity,normal load,filler,SWR1,SWR2,SWR3
0,sliding distance,sliding velocity,normal load,filler %,SWR,SWR,SWR
1,5,2,20,4,2.627179,17.275695,6.766977
2,5,4,60,2,3.173835,0.887598,3.631082
3,5,4,60,0,0.109748,0.768239,4.225313
4,5,4,20,0,1.481603,0.65849,12.264384


In [9]:
data = data.drop(0).reset_index(drop=True)


In [10]:
data.head()

,Material_Constant,sliding velocity,normal load,filler,SWR1,SWR2,SWR3
0,5,2,20,4,2.627179,17.275695,6.766977
1,5,4,60,2,3.173835,0.887598,3.631082
2,5,4,60,0,0.109748,0.768239,4.225313
3,5,4,20,0,1.481603,0.65849,12.264384
4,5,4,40,0,0.65849,0.740802,8.148819


In [12]:
import sklearn
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import  SVR
from sklearn.linear_model import  LinearRegression
from sklearn.tree import  DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [56]:
# Convert non-numeric columns to numeric (if any)
X = data.iloc[:, :-1].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN (from non-convertible strings)
X = X.drop(columns=["Material_Constant", "SWR1", "SWR2"], errors='ignore')  # Adjust columns as needed
y = data["SWR1"].astype(float)  # Ensure target is correctly aligned


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Regressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"Decision Tree Regression RMSE: {rmse:.4f}")


Decision Tree Regression RMSE: 3.2937


In [17]:
y.head(3)

0    2.627179
1    3.173835
2    0.109748
Name: SWR1, dtype: float64

In [57]:
model2 = SVR()
model2.fit(X_train, y_train)

# Predict and evaluate
y_pred = model2.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"SVR RMSE: {rmse:.4f}")


SVR RMSE: 3.3391


In [58]:
model3 = LinearRegression()
model3.fit(X_train, y_train)

# Predict and evaluate
y_pred = model3.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f" LinearRegression: {rmse:.4f}")


 LinearRegression: 2.7010


In [59]:
model4 =DecisionTreeRegressor()
model4.fit(X_train, y_train)

# Predict and evaluate
y_pred = model4.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"DecisionTreeRegressor: {rmse:.4f}")


DecisionTreeRegressor: 3.2937


In [60]:
from xgboost import XGBRegressor


model5=XGBRegressor()
model5.fit(X_train, y_train)

# Predict and evaluate
y_pred = model5.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"XGBRegressor: {rmse:.4f}")

XGBRegressor: 3.2814


In [61]:
#ANN
# Build ANN model
import tensorflow as tf
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

# Evaluate model
loss, mae = model.evaluate(X_test, y_test)
print(f"ANN Model MAE: {mae:.4f}")

# Predict
y_pred = model.predict(X_test)

Epoch 1/100


C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 14.0327 - mae: 3.1707 - val_loss: 16.5577 - val_mae: 3.4739
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 6.3140 - mae: 1.8860 - val_loss: 22.2740 - val_mae: 4.0222
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 8.2168 - mae: 2.0320 - val_loss: 24.8924 - val_mae: 4.2388
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 8.3745 - mae: 2.0694 - val_loss: 23.5492 - val_mae: 4.0909
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 7.1678 - mae: 1.8161 - val_loss: 19.9597 - val_mae: 3.8067
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 4.1682 - mae: 1.4863 - val_loss: 16.6654 - val_mae: 3.4337
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 6.4882 - mae: 2.0481 - val_loss: 15.3528 - val_mae: 3.1856
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 6.9968 - mae: 2.1928 - val_loss: 15.1554 - val_mae: 3.2029
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 6.5495 - mae: 2.0

In [62]:
# ANN
model6 = keras.Sequential([
	keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
	keras.layers.Dense(32, activation='relu'),
	keras.layers.Dense(16, activation='relu'),
	keras.layers.Dense(8, activation='relu'),
	keras.layers.Dense(1)  # Output layer for regression
])

# Compile model
model6.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model6.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

# Evaluate model
loss, mae = model6.evaluate(X_test, y_test)
print(f"ANN Model MAE: {mae:.4f}")


# Predict
y_pred = model6.predict(X_test)

Epoch 1/100


C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 6.1183 - mae: 1.6807 - val_loss: 15.5571 - val_mae: 3.2252
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 5.5862 - mae: 1.6907 - val_loss: 17.9478 - val_mae: 3.4405
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 6.2782 - mae: 1.7853 - val_loss: 18.3160 - val_mae: 3.4533
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 4.7138 - mae: 1.4833 - val_loss: 16.2220 - val_mae: 3.2582
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 4.2395 - mae: 1.5630 - val_loss: 14.6379 - val_mae: 3.0655
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 5.1189 - mae: 1.6940 - val_loss: 14.0152 - val_mae: 2.9659
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 5.9276 - mae: 1.8797 - val_loss: 13.8829 - val_mae: 2.9392
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 5.6852 - mae: 1.8520 - val_loss: 14.9844 - val_mae: 3.0623
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 4.0648 - mae: 1.47

In [63]:
def make_prediction(model, feature_values):
    input_df = pd.DataFrame([feature_values], columns=X.columns)
    prediction = model.predict(input_df)
    return prediction[0]

# Example usage with all models
sample_values = []
for i in range(len(X.columns)):
    value = float(input(f"Enter value for {X.columns[i]}: "))
    sample_values.append(value)
    
print("Predictions for sample input:", sample_values)
print(f"SVR: {make_prediction(model2, sample_values):.4f}")
print(f"Linear Regression: {make_prediction(model3, sample_values):.4f}")
print(f"Decision Tree: {make_prediction(model4, sample_values):.4f}")
print(f"XGBoost: {make_prediction(model5, sample_values):.4f}")

# ANN requires different handling
ann_prediction = model.predict(np.array(sample_values).reshape(1, -1))
print(f"ANN: {float(ann_prediction):.4f}")

dnn_prediction = model6.predict(np.array(sample_values).reshape(1, -1))
print(f"DNN: {float(dnn_prediction):.4f}")



Predictions for sample input: [2.0, 20.0, 4.0]
SVR: 2.3270
Linear Regression: 4.6284
Decision Tree: 7.5042
XGBoost: 7.5018
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
ANN: 2.8490
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
DNN: 2.6537


C:\Users\tejag\AppData\Local\Temp\ipykernel_11060\3760076337.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"ANN: {float(ann_prediction):.4f}")
C:\Users\tejag\AppData\Local\Temp\ipykernel_11060\3760076337.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"DNN: {float(dnn_prediction):.4f}")


In [65]:
X.head()


,sliding velocity,normal load,filler
0,2,20,4
1,4,60,2
2,4,60,0
3,4,20,0
4,4,40,0


In [64]:
y[:10]

0    2.627179
1    3.173835
2    0.109748
3    1.481603
4    0.658490
5    0.452712
6    2.582103
7    1.671841
8    7.020092
9    0.740802
Name: SWR1, dtype: float64

In [54]:
data.head()

,Material_Constant,sliding velocity,normal load,filler,SWR1,SWR2,SWR3
0,5,2,20,4,2.627179,17.275695,6.766977
1,5,4,60,2,3.173835,0.887598,3.631082
2,5,4,60,0,0.109748,0.768239,4.225313
3,5,4,20,0,1.481603,0.65849,12.264384
4,5,4,40,0,0.65849,0.740802,8.148819
